<a href="https://colab.research.google.com/github/KhamessiTaha/SIAMESE-Network_Face-recognition/blob/main/Train_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
import os

# Define the paths
rar_file_path = "/content/drive/My Drive/FACER/processed.rar"
extract_to_path = "/content/drive/My Drive/FACER/processed_images/"

# Create the directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Unzip the rar file to the specified directory
!unrar x -r "$rar_file_path" "$extract_to_path"


Streaming output truncated to the last 5000 lines.
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197601.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197602.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197603.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197604.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197605.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197606.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197607.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197608.jpg      97%  OK 
Extracting  /content/drive/My Drive/FACER/processed_images/processed/197609.jpg      97%  OK 
Extracting  /content/d

In [4]:
extracted_path = "/content/drive/My Drive/FACER/processed_images/"

# Check if the directory exists
if os.path.exists(extracted_path):
    print(f"The directory {extracted_path} exists.")
else:
    print(f"The directory {extracted_path} does not exist.")

# List the contents of the directory
files = os.listdir(extracted_path)
if files:
    print("Files in the directory:")
    for file in files[:10]:  # Print first 10 files as a sample
        print(file)
else:
    print("No files found in the directory.")

The directory /content/drive/My Drive/FACER/processed_images/ exists.
Files in the directory:
processed


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_siamese_model(input_shape):
    """
    Builds a Siamese Network model.

    Parameters:
    - input_shape: The shape of the input images.

    Returns:
    - model: The Siamese Network model.
    """

    # Define the base network
    def base_network(input_shape):
        inputs = tf.keras.Input(shape=input_shape)
        x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
        x = layers.MaxPooling2D()(x)
        x = layers.Conv2D(128, (3, 3), activation='relu')(x)
        x = layers.MaxPooling2D()(x)
        x = layers.Conv2D(128, (3, 3), activation='relu')(x)
        x = layers.Flatten()(x)
        x = layers.Dense(128, activation='relu')(x)
        return models.Model(inputs, x)

    input_shape = input_shape
    base_model = base_network(input_shape)

    input_a = tf.keras.Input(shape=input_shape)
    input_b = tf.keras.Input(shape=input_shape)

    processed_a = base_model(input_a)
    processed_b = base_model(input_b)


    distance = layers.Lambda(lambda tensors: tf.keras.backend.abs(tensors[0] - tensors[1]))([processed_a, processed_b])
    outputs = layers.Dense(1, activation='sigmoid')(distance)

    model = models.Model([input_a, input_b], outputs)
    return model

if __name__ == "__main__":
    input_shape = (100, 100, 3)
    model = build_siamese_model(input_shape)
    model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 model (Functional)          (None, 128)                  7448704   ['input_2[0][0]',             
                                                                     'input_3[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 128)                  0         ['model[0][0]',         

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split


# Paths
processed_images_path = "/content/drive/My Drive/FACER/processed_images/processed"

# Load preprocessed images and labels
def load_data(processed_images_path):
    images = []
    labels = []
    for img_file in os.listdir(processed_images_path):
        img_path = os.path.join(processed_images_path, img_file)
        img = load_img(img_path, target_size=(100, 100))
        img = img_to_array(img)
        images.append(img)
        label = int(img_file.split('.')[0])  # Use image_id as label
        labels.append(label)
    return np.array(images), np.array(labels)

# Create pairs of images and labels
def create_pairs(images, labels):
    pairs = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    class_indices = [np.where(labels == i)[0] for i in range(num_classes)]
    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]
        idx2 = np.random.choice(class_indices[label])
        positive_image = images[idx2]
        pairs += [[current_image, positive_image]]
        pair_labels += [1]
        negative_label = (label + np.random.randint(1, num_classes)) % num_classes
        idx2 = np.random.choice(class_indices[negative_label])
        negative_image = images[idx2]
        pairs += [[current_image, negative_image]]
        pair_labels += [0]
    return np.array(pairs), np.array(pair_labels)

if __name__ == "__main__":
    # Load and preprocess the data
    images, labels = load_data(processed_images_path)
    pairs, pair_labels = create_pairs(images, labels)
    pairs_train, pairs_val, labels_train, labels_val = train_test_split(pairs, pair_labels, test_size=0.2, random_state=42)

    input_shape = (100, 100, 3)
    model = build_siamese_model(input_shape)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit([pairs_train[:, 0], pairs_train[:, 1]], labels_train,
              validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val),
              batch_size=32,
              epochs=10)

    # Save the trained model
    model.save("/content/drive/My Drive/siamese_model.h5")
